# 지표 정리

#### 기본 세팅

In [13]:
import math
import numpy as np
import pandas as pd
import os
import glob

from pykrx import stock
from pykrx import bond

import time
from time import sleep
from datetime import datetime
from datetime import timedelta

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import psycopg2 as pg2
from sqlalchemy import create_engine

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


import warnings
warnings.filterwarnings('ignore')

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'


# BigQuery 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)


# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력

# Postgresql 연결
db_connect_info = pd.read_csv('key_value/db_connect_info.csv')
username = db_connect_info['username'][0]
password = db_connect_info['password'][0]
host = db_connect_info['host'][0]
database = db_connect_info['database'][0]
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:5432/{database}')



now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")

### 데이터 

In [52]:
glob.glob("**/*.csv", recursive=True)

['jupyter/ohlcv_df_raw.csv',
 'data_crawler/ohlcv_df_raw.csv',
 'data_crawler/sample.csv',
 'data_crawler/df_raw_total.csv',
 'data_crawler/kor_index_code_fundamental/kor_index_code_fundamental.csv',
 'data_crawler/index_code_master/index_code_master.csv',
 'data_crawler/kor_market_cap/kor_market_cap.csv',
 'data_crawler/snp500_daily/snp500_daily.csv',
 'data_crawler/bitcoin/bitcoin.csv',
 'data_crawler/usd_krw/usd_krw.csv',
 'data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.csv',
 'data_crawler/kor_ticker_list/kor_ticker_list_20240108.csv',
 'data_crawler/kor_ticker_list/kor_ticker_list.csv',
 'data_crawler/kor_index_list_df/kor_index_list_df.csv',
 'data_crawler/kor_index_ohlcv/kor_index_ohlcv.csv',
 'data_crawler/kor_stock_fundamental/kor_stock_fundamental.csv',
 'data_crawler/snp500_ticker_list/snp500_ticker_list.csv',
 'key_value/db_connect_info.csv',
 'key_value/naver_apikey.csv',
 'key_value/chatgpt_apikey.csv']

In [53]:
glob.glob("**/*.parquet", recursive=True)

['jupyter/df.parquet',
 'jupyter/kor_stock_ohlcv.parquet',
 'data_crawler/cleaning/bitcoin/bitcoin_anal_cleaning.parquet',
 'data_crawler/cleaning/bitcoin/bitcoin_cleaning.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_anal_kosdaq.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_kosdaq.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/buy_sell_count_kosdaq.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_anal_kospi.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_kospi.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/buy_sell_count_kospi.parquet',
 'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_cleaning.parquet',
 'data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_anal_cleaning.parquet',
 'data_crawler/kor_index_code_fundamental/kor_index_code_fundamental.parquet',
 'data_crawler/kor_index_list_df/kor_index_list_df.parquet',
 'data_crawler/kor_index_ohlcv/kor_index_ohlcv.parquet']

In [61]:
# 코스피 지표
t = pq.read_table('data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_kospi.parquet')
kor_stock_ohlcv_kospi = t.to_pandas()

# 코스피 매수/매도 지표
t = pq.read_table('data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_anal_kospi.parquet')
kor_stock_ohlcv_anal_kospi = t.to_pandas()

# 코스피 주식별 매수매도 지표
t = pq.read_table('data_crawler/cleaning/kor_stock_ohlcv/buy_sell_count_kospi.parquet')
buy_sell_count_kospi = t.to_pandas()

# 인덱스 지표
t = pq.read_table('data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_cleaning.parquet')
kor_index_ohlcv_cleaning = t.to_pandas()

# 인덱스 매수/매도 지표
t = pq.read_table('data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_anal_cleaning.parquet')
kor_index_ohlcv_anal_cleaning = t.to_pandas()

# 인덱스 PBR
t = pq.read_table('data_crawler/kor_index_code_fundamental/kor_index_code_fundamental.parquet')
kor_index_code_fundamental = t.to_pandas()


index_code_master = pd.read_csv('data_crawler/index_code_master/index_code_master.csv', dtype = {'ticker':str, 
                                                                                                'index_code':str})


not_sectors = ["1002","1003","1004","1028","1034","1035","1150","1151",
           "1152","1153","1154","1155","1156","1157","1158","1159",
           "1160","1167","1182","1224","1227","1232","1244","1894",
           "2002","2003","2004","2181","2182","2183","2184","2189",
           "2203","2212","2213","2214","2215","2216","2217","2218"]

index_code_master = index_code_master[~index_code_master['index_code'].isin(not_sectors)].reset_index(drop = True)


In [45]:
kor_index_code_fundamental.head()

,date,close,price_change_percentage,per,porward_per,pbr,dividend_yield,index_code
0,2018-01-02,2479.65,0.49,13.06,0.0,1.12,1.35,1001
1,2018-01-03,2486.35,0.27,13.09,0.0,1.12,1.35,1001
2,2018-01-04,2466.46,-0.80,12.99,0.0,1.11,1.36,1001
3,2018-01-05,2497.52,1.26,13.15,0.0,1.12,1.34,1001
4,2018-01-08,2513.28,0.63,13.23,0.0,1.13,1.34,1001


In [55]:
index_code_master

,ticker,index_code,index_code_nm,market,corp_name
0,5930,1001,코스피,KOSPI,삼성전자
1,660,1001,코스피,KOSPI,SK하이닉스
2,373220,1001,코스피,KOSPI,LG에너지솔루션
3,207940,1001,코스피,KOSPI,삼성바이오로직스
4,5490,1001,코스피,KOSPI,POSCO홀딩스
...,...,...,...,...,...
13730,122870,2218,코스닥 150 커뮤니케이션서비스,KOSDAQ,와이지엔터테인먼트
13731,119860,2218,코스닥 150 커뮤니케이션서비스,KOSDAQ,커넥트웨이브
13732,78340,2218,코스닥 150 커뮤니케이션서비스,KOSDAQ,컴투스
13733,95660,2218,코스닥 150 커뮤니케이션서비스,KOSDAQ,네오위즈


In [37]:
kor_stock_ohlcv_kospi.head(2)

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name,market,MA5,MA20,MA60,MA120,upper,lower,MACD_DIFF,MACD,MACD_Signal,RSI
0,2023-01-09 00:00:00+00:00,59700,60700,59600,60700.0,18640107,2.881356,005930,삼성전자,KOSPI,58220.0,58315.0,59373.333333,58841.666667,61407.749586,55222.250414,297.792250,-382.242616,-680.034867,59.579300
1,2023-01-10 00:00:00+00:00,60200,61100,59900,60400.0,14859797,-0.494234,005930,삼성전자,KOSPI,59220.0,58360.0,59436.666667,58829.166667,61545.215848,55174.784152,402.537204,-176.863361,-579.400566,58.001471


In [38]:
kor_stock_ohlcv_anal_kospi.head(2)

,date,ticker,corp_name,market,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2023-01-09 00:00:00+00:00,005930,삼성전자,KOSPI,60700,-,-,-,-,-,-
1,2023-01-10 00:00:00+00:00,005930,삼성전자,KOSPI,60400,골든크로스(매수),-,-,-,-,-


In [39]:
buy_sell_count_kospi.head(2)

,date,ticker,corp_name,market,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2024-01-05 00:00:00+00:00,005930,삼성전자,KOSPI,76600,-,-,정배열(매수),-,-,-
1,2024-01-05 00:00:00+00:00,000660,SK하이닉스,KOSPI,137500,-,-,정배열(매수),-,-,-


In [40]:
kor_index_ohlcv_cleaning.head(2)

,date,open,high,low,close,volume,trading_value,market_cap,index_code,index_code_nm,...,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI,MA5-20,MA20-60,MA60-120
0,2023-01-02 00:00:00+00:00,2249.95,2259.88,2222.37,2225.67,346344799,5200137586818,1759241799519040,1001,코스피,...,-19.659065,-10.73,0.0,10.73,6.386393,15.439962,29.260008,-66.4275,-0.274167,-23.95300
1,2023-01-03 00:00:00+00:00,2230.98,2230.98,2180.67,2218.68,410245325,6149082624890,1753771077018843,1001,코스피,...,-23.522861,-6.99,0.0,6.99,5.930222,14.836394,28.556518,-76.0875,-10.070167,-23.17575


In [41]:
kor_index_ohlcv_anal_cleaning.head(2)

,date,index_code,index_code_nm,market,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2023-01-02 00:00:00+00:00,1001,코스피,KOSPI,2225.67,-,데드크로스(매도),역배열(매도),-,-,RSI 하단 상향 돌파(매수)
1,2023-01-03 00:00:00+00:00,1001,코스피,KOSPI,2218.68,-,-,역배열(매도),-,-,-


### 상황 단일주가 (삼성전자

- trading 추가

In [47]:
ticker_nm = '005930'

#### 주가 등락률

In [169]:
kor_stock_ohlcv_kospi_sample = kor_stock_ohlcv_kospi[kor_stock_ohlcv_kospi['ticker'] == ticker_nm]

In [171]:
df1_set = kor_stock_ohlcv_kospi_sample[['ticker', 'date', 'close', 'corp_name']]
df1_set = df1_set[df1_set['date'] == '2024-01-05']

df1 =  kor_stock_ohlcv_kospi_sample[kor_stock_ohlcv_kospi_sample['date'] > set_date].groupby("ticker")['date'].min().reset_index()

diff_date_list = [30, 90, 180, 240, 365]
for diff_date in diff_date_list:
# diff_date = 240
    now = datetime.now()
    now = now + timedelta(days=-diff_date)
    set_date = now.strftime('%Y-%m-%d')
    df1 =  kor_stock_ohlcv_kospi_sample[kor_stock_ohlcv_kospi_sample['date'] > set_date].groupby("ticker")['date'].min().reset_index()

    
    df1 = df1[['ticker','date']]
    df1 = pd.merge(df1, kor_stock_ohlcv_kospi_sample[['ticker', 'date', 'close']],
                  on = ['ticker', 'date'], 
                  how = 'left')
    
    df1.columns = ['ticker', f'date_{diff_date}', f'close_{diff_date}']

    df1_set = pd.merge(df1_set, df1, 
                      on = 'ticker', 
                      how = 'left')

    df1_set[f'per_{diff_date}'] = (df1_set['close'] - df1_set[f'close_{diff_date}']) / df1_set[f'close_{diff_date}'] * 100

df1_set_2 = df1_set[['date','ticker','per_30', 'per_90', 'per_180', 'per_240', 'per_365', 'corp_name']]
df1_set_2

df1_set_3 = pd.melt(df1_set_2, 
        id_vars= ['date', 'ticker', 'corp_name'], 
        value_vars=['per_30', 'per_90', 'per_180', 'per_240','per_365'])    

stock_per_set = df1_set_3
stock_per_set

,date,ticker,corp_name,variable,value
0,2024-01-05 00:00:00+00:00,005930,삼성전자,per_30,4.217687
1,2024-01-05 00:00:00+00:00,005930,삼성전자,per_90,12.647059
2,2024-01-05 00:00:00+00:00,005930,삼성전자,per_180,4.502046
3,2024-01-05 00:00:00+00:00,005930,삼성전자,per_240,17.125382
4,2024-01-05 00:00:00+00:00,005930,삼성전자,per_365,26.611570


#### 주가 보조지표 매수/매도 

In [172]:
dfdf = kor_stock_ohlcv_anal_kospi[kor_stock_ohlcv_anal_kospi['ticker'] == ticker_nm].tail(1)

In [185]:
# pd.wide_to_long(dfdf, stubnames='ht', i=['date', 'ticker','corp_name','market','close'], j='age')
dfdf2 = pd.melt(dfdf, 
        id_vars= ['date', 'ticker', 'corp_name'], 
        value_vars=['5_20_cross', '20_60_cross', 'array', 'Bollinger_band','MACD','RSI'])
stock_indicator = dfdf2[dfdf2['value'] != '-']
stock_indicator

,date,ticker,corp_name,variable,value
2,2024-01-05 00:00:00+00:00,005930,삼성전자,array,정배열(매수)


#### 주가 매칭 인덱스

In [65]:
ticker_index_code_df = index_code_master[index_code_master['ticker'] == ticker_nm].reset_index(drop = True)
ticker_index_code_df

,ticker,index_code,index_code_nm,market,corp_name
0,005930,1001,코스피,KOSPI,삼성전자
1,005930,1013,전기전자,KOSPI,삼성전자
2,005930,1027,제조업,KOSPI,삼성전자


#### 인덱스 등락률

In [178]:
kor_index_ohlcv_cleaning_sample = kor_index_ohlcv_cleaning[kor_index_ohlcv_cleaning['index_code'].isin(ticker_index_code_df['index_code'])]

In [180]:
# df1_set =  ohlcv_df_raw[ohlcv_df_raw['date'] > '2023-10-01'].groupby("ticker")['date'].max().reset_index()
df1_set =  kor_index_ohlcv_cleaning_sample.groupby("index_code")['date'].max().reset_index()
df1_set = df1_set[['index_code', 'date']]
df1_set = pd.merge(df1_set, kor_index_ohlcv_cleaning_sample[['index_code', 'date', 'close', 'index_code_nm']],
              on = ['index_code', 'date'], 
              how = 'left')



diff_date_list = [30, 90, 180, 240, 365]
for diff_date in diff_date_list:
# diff_date = 240
    now = datetime.now()
    now = now + timedelta(days=-diff_date)
    set_date = now.strftime('%Y-%m-%d')
    df1 =  kor_index_ohlcv_cleaning_sample[kor_index_ohlcv_cleaning_sample['date'] > set_date].groupby("index_code")['date'].min().reset_index()

    
    df1 = df1[['index_code','date']]
    df1 = pd.merge(df1, kor_index_ohlcv_cleaning_sample[['index_code', 'date', 'close']],
                  on = ['index_code', 'date'], 
                  how = 'left')
    
    df1.columns = ['index_code', f'date_{diff_date}', f'close_{diff_date}']

    df1_set = pd.merge(df1_set, df1, 
                      on = 'index_code', 
                      how = 'left')

    df1_set[f'per_{diff_date}'] = (df1_set['close'] - df1_set[f'close_{diff_date}']) / df1_set[f'close_{diff_date}'] * 100

df1_set_2 = df1_set[['date','index_code','per_30', 'per_90', 'per_180', 'per_240', 'per_365', 'index_code_nm']]
df1_set_2

df1_set_3 = pd.melt(df1_set_2, 
        id_vars= ['date', 'index_code', 'index_code_nm'], 
        value_vars=['per_30', 'per_90', 'per_180', 'per_240','per_365'])    
index_per = df1_set_3.sort_values(by = ['index_code'])
index_per.head()

,date,index_code,index_code_nm,variable,value
0,2024-01-05 00:00:00+00:00,1001,코스피,per_30,1.688578
3,2024-01-05 00:00:00+00:00,1001,코스피,per_90,4.964273
6,2024-01-05 00:00:00+00:00,1001,코스피,per_180,-1.562428
9,2024-01-05 00:00:00+00:00,1001,코스피,per_240,3.944780
12,2024-01-05 00:00:00+00:00,1001,코스피,per_365,9.262438


#### 인덱스 보조지표 매수/매도

In [166]:
kor_index_ohlcv_anal_cleaning_df = kor_index_ohlcv_anal_cleaning[kor_index_ohlcv_anal_cleaning['index_code'].isin(ticker_index_code_df['index_code'])]


In [191]:
max_date = max(kor_index_ohlcv_anal_cleaning_df['date'])
kor_index_ohlcv_anal_cleaning_df_22 = kor_index_ohlcv_anal_cleaning_df[kor_index_ohlcv_anal_cleaning_df['date'] == max_date]


,date,index_code,index_code_nm,market,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
248,2024-01-05 00:00:00+00:00,1001,코스피,KOSPI,2578.08,-,-,-,-,하향돌파(매도),-
3236,2024-01-05 00:00:00+00:00,1013,전기전자,KOSPI,27597.62,-,-,-,-,-,-
6224,2024-01-05 00:00:00+00:00,1027,제조업,KOSPI,6944.05,-,-,-,-,-,-


In [184]:
# pd.wide_to_long(dfdf, stubnames='ht', i=['date', 'ticker','corp_name','market','close'], j='age')
kor_index_ohlcv_anal_cleaning_df_2 = pd.melt(kor_index_ohlcv_anal_cleaning_df_22, 
        id_vars= ['date', 'index_code', 'index_code_nm'], 
        value_vars=['5_20_cross', '20_60_cross', 'array', 'Bollinger_band','MACD','RSI'])

index_indicator = kor_index_ohlcv_anal_cleaning_df_2[kor_index_ohlcv_anal_cleaning_df_2['value'] != '-']

#### 최종

In [199]:
df1 = pd.concat([stock_per_set,stock_indicator])
df1.columns = ['date', 'code', 'code_nm', 'variable', 'value']

In [200]:
df2 = pd.concat([index_per,index_indicator])
df2.columns = ['date', 'code', 'code_nm', 'variable', 'value']

In [235]:
df3 = pd.concat([df1,df2]).reset_index(drop = True)

In [236]:
df4 = df3[['code_nm', 'variable', 'value']]

In [237]:
df4

,code_nm,variable,value
0,삼성전자,per_30,4.217687
1,삼성전자,per_90,12.647059
2,삼성전자,per_180,4.502046
3,삼성전자,per_240,17.125382
4,삼성전자,per_365,26.61157
5,삼성전자,array,정배열(매수)
6,코스피,per_30,1.688578
7,코스피,per_90,4.964273
8,코스피,per_180,-1.562428
9,코스피,per_240,3.94478


In [238]:
df3['date'][0]

Timestamp('2024-01-05 00:00:00+0000', tz='UTC')

'\n- 날짜:2024-01-05 \ncode_nm\tvariable\tvalue\n0\t삼성전자\tper_30\t4.217687\n1\t삼성전자\tper_90\t12.647059\n2\t삼성전자\tper_180\t4.502046\n3\t삼성전자\tper_240\t17.125382\n4\t삼성전자\tper_365\t26.61157\n5\t삼성전자\tarray\t정배열(매수)\n6\t코스피\tper_30\t1.688578\n7\t코스피\tper_90\t4.964273\n8\t코스피\tper_180\t-1.562428\n9\t코스피\tper_240\t3.94478\n10\t코스피\tper_365\t9.262438\n11\t전기전자\tper_30\t3.334085\n12\t전기전자\tper_90\t6.763262\n13\t전기전자\tper_180\t-2.191732\n14\t전기전자\tper_240\t9.321707\n15\t전기전자\tper_365\t18.793198\n16\t제조업\tper_30\t2.828656\n17\t제조업\tper_90\t4.890103\n18\t제조업\tper_180\t-3.356877\n19\t제조업\tper_240\t4.383504\n20\t제조업\tper_365\t13.490275\n21\t코스피\tMACD\t하향돌파(매도)\n\n\n보고서 형태로 요약해줘 \n'

### gemini 

In [197]:
import google.generativeai as genai

api_key_df = pd.read_csv('key_value/chatgpt_apikey.csv')
GOOGLE_API_KEY = api_key_df[api_key_df['corp'] == 'google'].reset_index()['api_key'][0]


genai.configure(api_key=GOOGLE_API_KEY)

# Set up the model
generation_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 2048,
}

model = genai.GenerativeModel('gemini-pro',
                             generation_config=generation_config)

In [242]:
date_nm = df3['date'][0].strftime('%Y-%m-%d')

prompt = f"""
- 날짜:{date_nm} 
code_nm	variable	value
0	삼성전자	per_30	4.217687
1	삼성전자	per_90	12.647059
2	삼성전자	per_180	4.502046
3	삼성전자	per_240	17.125382
4	삼성전자	per_365	26.61157
5	삼성전자	array	정배열(매수)
6	코스피	per_30	1.688578
7	코스피	per_90	4.964273
8	코스피	per_180	-1.562428
9	코스피	per_240	3.94478
10	코스피	per_365	9.262438
11	전기전자	per_30	3.334085
12	전기전자	per_90	6.763262
13	전기전자	per_180	-2.191732
14	전기전자	per_240	9.321707
15	전기전자	per_365	18.793198
16	제조업	per_30	2.828656
17	제조업	per_90	4.890103
18	제조업	per_180	-3.356877
19	제조업	per_240	4.383504
20	제조업	per_365	13.490275
21	코스피	MACD	하향돌파(매도)


보고서 형태로 설명식으로 요약해줘. 
"""

response = model.generate_content(prompt)
print(response.text)

'\n- 날짜:2024-01-05 \ncode_nm\tvariable\tvalue\n0\t삼성전자\tper_30\t4.217687\n1\t삼성전자\tper_90\t12.647059\n2\t삼성전자\tper_180\t4.502046\n3\t삼성전자\tper_240\t17.125382\n4\t삼성전자\tper_365\t26.61157\n5\t삼성전자\tarray\t정배열(매수)\n6\t코스피\tper_30\t1.688578\n7\t코스피\tper_90\t4.964273\n8\t코스피\tper_180\t-1.562428\n9\t코스피\tper_240\t3.94478\n10\t코스피\tper_365\t9.262438\n11\t전기전자\tper_30\t3.334085\n12\t전기전자\tper_90\t6.763262\n13\t전기전자\tper_180\t-2.191732\n14\t전기전자\tper_240\t9.321707\n15\t전기전자\tper_365\t18.793198\n16\t제조업\tper_30\t2.828656\n17\t제조업\tper_90\t4.890103\n18\t제조업\tper_180\t-3.356877\n19\t제조업\tper_240\t4.383504\n20\t제조업\tper_365\t13.490275\n21\t코스피\tMACD\t하향돌파(매도)\n\n\n보고서 형태로 설명식으로 요약해줘. \n'

**보고서**

**날짜:** 2024년 1월 5일

**주제:** 주식 시장 분석

**내용:**

삼성전자의 주가는 최근 30일 동안 4.21% 상승했고, 90일 동안은 12.64% 상승했으며, 180일 동안은 4.50% 상승했으며, 240일 동안은 17.12% 상승했으며, 365일 동안은 26.61% 상승했습니다. 해당 기간 동안 삼성전자는 정배열(매수)로 분석되었습니다.

코스피 지수는 최근 30일 동안 1.68% 상승했고, 90일 동안은 4.96% 상승했으며, 180일 동안은 -1.56% 하락했으며, 240일 동안은 3.94% 상승했으며, 365일 동안은 9.26% 상승했습니다. 코스피 지수의 MACD는 하향 돌파(매도)로 나타났습니다.

전기전자 산업의 주가는 최근 30일 동안 3.33% 상승했고, 90일 동안은 6.76% 상승했으며, 180일 동안은 -2.19% 하락했으며, 240일 동안은 9.32% 상승했으며, 365일 동안은 18.79% 상승했습니다.

제조업 산업의 주가는 최근 30일 동안 2.82% 상승했고, 90일 동안은 4.89% 상승했으며, 180일 동안은 -3.35% 하락했으며, 240일 동안은 4.38% 상승했으며, 365일 동안은 13.49% 상승했습니다.

**결론:**

전반적으로 삼성전자, 코스피 지수, 전기전자 산업, 제조업 산업의 주가는 최근 상승세를 보였습니다. 그러나 코스피 지수의 MACD가 하향 돌파로 나타나 앞으로 주가가 하락할 가능성이 있으므로 주의가 필요합니다.
